In [ ]:
#setup
pip install -r
pip install langchain langchain-text-splitters langchain-community bs4
pip install -U "langchain-openai"

In [ ]:
# Installer les dependencies
import os
from google import genai
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver  

**Création de la clé API**

In [ ]:
# Set your Gemini API key
export GOOGLE_API_KEY=
os.environ['GOOGLE_API_KEY'] = 

***1.Solution 1 - Le builtin de Gemini***

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1-mini")

tool = {"type": "web_search"}
model_with_tools = model.bind_tools([tool])

response = model_with_tools.invoke("What was a positive news story from today?")
response.content_blocks

***2. Solution 2 - Construction pas à pas***

***Create an Agent***

The easiest way to get started with a standalone model in LangChain is to use init_chat_model to initialize one from a chat model provider of your choice (examples below):

In [4]:
#initialize the model
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

#create an agent
agent = create_agent(
    "2.5-flash-lite",
    tools=tools
    checkpointer=InMemorySaver(), 
)
agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)




NameError: name 'genai' is not defined

***Initialize a model***

In [8]:
client = genai.Client()
# Choose a tool
tools=genai.types.Tool(function_declarations=[get_matches_declaration])
tool = {"type": "web_search"}
model_with_tools = model.bind_tools([tool])

NameError: name 'genai' is not defined

***Tools***

Many AI applications interact with users via natural language. However, some use cases require models to interface directly with external systems—such as APIs, databases, or file systems—using structured input.
Tools are components that agents call to perform actions. They extend model capabilities by letting them interact with the world through well-defined inputs and outputs. Tools encapsulate a callable function and its input schema. These can be passed to compatible chat models, allowing the model to decide whether to invoke a tool and with what arguments. In these scenarios, tool calling enables models to generate requests that conform to a specified input schema.

In [ ]:
#The first step is to describe the function so the LLM understands what it does.
get_matches_declaration = {
    "name": "get_matches",
    "description": "Return the rows in a DataFrame about men's football games which satisfy the criteria",
    "parameters": {
        "type": "object",
        "properties": {
            "nationality_name": {
                "type": "string",
                "description": "The name of the player's country ",
            },
            "club_contract_valid_until_year": {
                "type": "number",
                "description": "When the player is up to be bought",
            },
            "club_position": {
                "type": "number",
                "description": "The field position",
            },
        },
        "required": ["nationality_name", "club_contract_valid_until_year", "club_position"],
    },
}

Then, we can feed the function declaration together with the user's query to the LLM:

In [ ]:
# Wrap the declaration into a `Tool`
tools = genai.types.Tool(
    function_declarations=[
        get_matches_declaration
    ]
)

user_question = """Tell me more about the player, his availability"""

# Use the model
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_question,  # The question
    config=genai.types.GenerateContentConfig(
        tools=[tools]        # The tool
    )
)

In [ ]:
#alternative : deuxième fichier fastapi créer l'appel à la fonction
@tool("web_search")  # Custom name
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

***Create a message***

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.
Messages are objects that contain:
 Role - Identifies the message type (e.g. system, user)
 Content - Represents the actual content of the message (like text, images, audio, documents, etc.)
 Metadata - Optional fields such as response information, message IDs, and token usage

In [5]:
# Générer une réponse
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_question,
    config=genai.types.generateContentConfig(tools=[tools]),)

NameError: name 'client' is not defined

In [ ]:
#The LLM has extracted the arguments that we can use for our function:
args = response.candidates[0].content.parts[0].function_call.args
args

In [6]:
system_msg = SystemMessage("You are a Football expert")
human_msg = HumanMessage("Hello, how are you? Want to talk about outstanding players ?")

# Use with chat models
messages = [system_msg, human_msg]
response = model.invoke(messages)  # Returns AIMessage

NameError: name 'SystemMessage' is not defined

***/***

In [ ]:
Markdown(response.text)